# Photometric monitoring

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob as glob
import matplotlib as mpl
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import numpy as np
import pandas as pd
import seaborn as sns

import corner
import json
import pathlib
import pickle
import utils
import warnings
import sys

from astropy import constants as const
from astropy import units as uni
from astropy.io import ascii, fits
from astropy.time import Time
from mpl_toolkits.axes_grid1 import ImageGrid

# Default figure dimensions
FIG_WIDE = (11, 5)
FIG_LARGE = (8, 11)

# Figure style
sns.set(style="ticks", palette="colorblind", color_codes=True, context="talk")
params = utils.plot_params()
plt.rcParams.update(params)

## [Dowload data](https://www.dropbox.com/sh/74sihxztgd82jjz/AADgB_f5RYc3De3IEioUGAfha?dl=1)

Unzip this into a folder named `data` in the same level as this notebook

## Load

In [3]:
dirpath = "../data/photometric"

mid_transit_times = {
    "Transit 1": "2013-12-19 12:00:00",
    "Transit 2": "2015-09-27 12:00:00",
    "Transit 3": "2016-12-11 12:00:00",
}

# Load processed data
df_stell_data = pd.read_csv(
    f"{dirpath}/AP37847073.csv",
#     names=["t_HJD", "t_UT", "f"],
#     parse_dates=[1],
#     infer_datetime_format=True,
)

# Load model data
# df_stell_model = pd.read_csv(
#     f"{dirpath}/HATP23_GP_model_Prot7_v3.csv", names=["t_HJD", "f", "f_err"]
# )

df_stell_data.sort_values("hjd", inplace=True)
df_stell_data

,hjd,camera,filter,mag,mag err,flux (mJy),flux err
8,2.456653e+06,bd,V,11.743,0.02,76.459,1.407
235,2.456657e+06,bd,V,11.736,0.02,76.977,1.416
164,2.456675e+06,bd,V,11.743,0.02,76.461,1.407
98,2.456883e+06,bd,V,11.717,0.02,78.340,1.441
238,2.456888e+06,bd,V,11.740,0.02,76.677,1.411
...,...,...,...,...,...,...,...
51,2.458431e+06,bd,V,11.750,0.02,76.012,1.399
35,2.458435e+06,bd,V,11.753,0.02,75.756,1.394
19,2.458446e+06,bd,V,11.743,0.02,76.487,1.407
121,2.458451e+06,bd,V,11.713,0.02,78.583,1.446


## Plot

In [16]:
np.diff(t)

array([4.0135800e+00, 1.7983000e+01, 2.0833067e+02, 4.8757600e+00,
       2.0488200e+00, 1.9816000e+00, 3.0745700e+00, 2.9084600e+00,
       2.0065500e+00, 2.9958400e+00, 1.9929400e+00, 2.9814300e+00,
       2.9577100e+00, 8.0552200e+00, 9.8811000e-01, 1.0822000e+00,
       2.9261900e+00, 1.9041160e+01, 9.0541000e-01, 2.3876290e+01,
       3.0792700e+00, 2.9964200e+00, 5.9923500e+00, 2.0104200e+00,
       1.9731100e+00, 2.0006500e+00, 2.8602900e+00, 9.8327000e-01,
       1.2141020e+01, 1.0201600e+00, 9.6498000e-01, 4.9889600e+00,
       2.1998050e+01, 9.7421000e-01, 2.9783900e+00, 1.0059000e+00,
       2.9726700e+00, 2.0109900e+00, 4.9985700e+00, 5.9923600e+00,
       1.6034729e+02, 3.0989170e+01, 2.9623200e+00, 1.1720500e+01,
       7.9668300e+00, 2.0216400e+00, 2.9208300e+00, 6.1750600e+00,
       1.7030000e-01, 1.8120800e+00, 3.9516400e+00, 1.0245900e+00,
       2.8318700e+00, 1.1868300e+00, 8.0824000e-01, 3.1329700e+00,
       1.8551600e+00, 1.3145820e+01, 3.2165000e+00, 4.8445700e

In [35]:
t, f = df_stell_data["hjd"].to_numpy() - 2.45e6, df_stell_data["mag"].to_numpy()
ferr = df_stell_data["mag err"].to_numpy() * np.ones_like(f)
t_binned, f_binned, f_err_binned = BinnedLC(
    t, f, ferr, diff_lim=7 #diff_lim=np.mean(np.diff(t_binned))
)

fig, ax = plt.subplots(figsize=FIG_WIDE)

ax.plot(t, f, "ro", alpha=0.5, mew=0)
# ax.plot(df_stell_model["t_HJD"], df_stell_model["f"], color="grey")
# f_d = df_stell_model["f"] - df_stell_model["f_err"]
# f_u = df_stell_model["f"] + df_stell_model["f_err"]
# ax.fill_between(df_stell_model["t_HJD"], f_d, f_u, alpha=0.3, lw=0, color="grey")

p_kwargs = {"ls": "--", "c": "darkgrey", "lw": 1.0}
trans = transforms.blended_transform_factory(ax.transData, ax.transAxes)

for transit_name, t0 in mid_transit_times.items():
    t_mid = Time(t0).jd - 2.45e6
    ax.axvline(t_mid, **p_kwargs)
    ax.annotate(
        transit_name,
        xy=(t_mid, 0.1),
        xycoords=trans,
        ha="right",
        rotation=90.0,
        fontsize=12,
    )
    
ax.plot(t_binned, f_binned, "b.")

# Save
#ax.set_ylim(0.88, 0.98)
ax.set_xlabel("Date (HJD - 2.45e6)")
ax.set_ylabel("Mag")
fig.tight_layout()
fig.set_size_inches(FIG_WIDE)
#utils.savefig("../paper/figures/photometric_act/phot_mon_full.pdf")

In [ ]:
plt.plot()

In [4]:
%matplotlib qt5

In [6]:
triangular_number = lambda n: int(n)*(int(n)+1)//2 #this is addition factorial. i.e X+(X-1)+(X-2)+(X-3)...

def weighted_avg_and_std(values, weights):
    """ Return the weighted average and standard deviation. values, weights -- Numpy ndarrays with the same shape."""
    average = np.average(values, weights=weights) #weights = 1./err_bar^2. Where err_bar=std & err_bar^2 = variance
    variance = np.average((values-average)**2, weights=weights) # Fast and numerically precise
    return average, np.sqrt(variance)

# to code that averages the datapoints that are close in time
def BinnedLC(time, flux, flx_err, diff_lim=.1): # diff_lim = max difference in time [days]. 0.1 = 2.4hrs
    #To block togther all datapoints that have similar times stamps
    DiffGroupTotal = [] #This will be a list of list, where each list holds the indeces that should be combined 
    DiffGroup_i = [] #To keep track of elements that should be combined
    Start_i, end_i = 0, 1 #The 1st index of DiffGroup_i. end_i is initalized to Start_i+1
    for d in range(0,len(time)-1):
        if time[d+1]-time[Start_i]<diff_lim: #If the 1st element (Start_i) and the last element searched (end_i) have a time difference less than 'diff_lim,' add those indeces
            if len(DiffGroup_i) == 0: #Starting new group cointaining small time differences amongst them
                DiffGroup_i.append(Start_i) #add start to the list of close times
                DiffGroup_i.append(d+1)
                Start_i = d #To keep track of where DiffGroup_i started
            else: #if DiffGroup_i already initialized, add next element to our list
                DiffGroup_i.append(d+1)
        else:
            if len(DiffGroup_i) > 1: # if the previous step was the last step in the grouped times, append the DiffGroup_i to DiffGroupTotal
                DiffGroupTotal.append(DiffGroup_i)
            if time[d+1]-time[d]> diff_lim and d not in DiffGroup_i: #only add individual element if not going to go
                DiffGroupTotal.append([d]) #in the next list
            Start_i= d+1 #reset Start_i
            DiffGroup_i = [] #rest the group list
    if len(DiffGroup_i) > 1: # For the last element
        DiffGroupTotal.append(DiffGroup_i)
    else:
        DiffGroupTotal.append([d+1])
    
    #Extra checks to make sure all the indecies are accounted for in the right sub-lists:
    TotalSum = 0 #to check if adds
    for i in range(len(DiffGroupTotal)): #To scan each sublist
        sublist = DiffGroupTotal[i]
        FirstTime, LastTime = time[sublist[0]], time[sublist[-1]]
        if LastTime-FirstTime > diff_lim: #To make sure the differences of the indeces I marked in each subdirectory are indeed within the approporate limit
            sys.exit("In sublist "+str(i)+", the 1st and last times are "+str(FirstTime)+","+str(LastTime)+". Their difference is greater than diff_lim!!!!")
        TotalSum += np.sum(sublist)
    Triangle = triangular_number(len(time)-1) #triangle number of len(time)-1, to makes sure every index is accounted for
    if TotalSum != Triangle:
        print ("Sum of all indeces is "+str(TotalSum)+". However, the triangle number of the inputed time array length is "+str(Triangle)+".") 
        print ("DiffGroupTotal", DiffGroupTotal)
        sys.exit()
        
    #To weight average all blocked data
    new_time, new_flux, new_FlxErr = [], [], []
    for g in DiffGroupTotal:
        if len(g) == 1:
            new_time.append(time[g][0]),  new_flux.append(flux[g][0]), new_FlxErr.append(flx_err[g][0])
        else:
            Flux, STD = weighted_avg_and_std(flux[g], 1./(flx_err[g]**2))
            new_time.append(np.mean(time[g])),  new_flux.append(Flux), new_FlxErr.append(STD)
    
    return np.array(new_time, dtype="object"), np.array(new_flux, dtype="object"), np.array(new_FlxErr, dtype="object") #need the "object" because each array is of different size